# Fine tune Intent Model with Hugging Faces & Open vino 

#### Load dataset from downloads 📥📥

In [1]:
!pip install transformers datasets evaluate accelerate 


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import os
labeled_dataset = "labeled_dataset.csv"

# Assuming the file is in the current working directory
df = pd.read_csv(labeled_dataset)

# Display the first few rows of the dataframe
df.head()

In [ ]:
test_size = 0.2 ## Define the testing size for metrics
number_of_labels = os.environ.get('number_labels', 6)
label_column_name = 'sentiment'
text_column_name = os.environ.get('prompt_column', 'question')
model_name = "intent"

In [ ]:
print(f"The number of labels to be showed are: {number_of_labels} with a label colum name: {label_column_name} and a prompt column name: {text_column_name} (The test size is {test_size})")

## Define mappings 

Here you have to define a map so the model can be properly trained lets see an example

```python
category_to_label = {
    'availability': 0,
    'irrelevant': 1,
    'post sale': 2,
    'invoice':3,
    'service':4,
    'pricing':5,
    'general':6,
    'cancelation policy':7,
    'cancel reservation':8
}
```

In [ ]:
### For now the map is being defined within the notebook

category_to_label={
 'inventory': 0,
 'checkout': 1,
 'irrelevant': 2,
 'conversational': 3,
 'feedback': 4,
 'postSale': 5
}

# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[label_column_name].replace(category_to_label)
df = df.drop('annotation_id',axis=1)
df = df.drop('annotator',axis=1)
df = df.drop('created_at',axis=1)
df = df.drop('id',axis=1)
df = df.drop('lead_time',axis=1)
df = df.drop('updated_at',axis=1)

In [ ]:
category_to_label

In [ ]:
df.head(3)

In [ ]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(df, test_size=test_size)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
from transformers import AutoTokenizer
base_model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples[text_column_name], truncation=True)

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=number_of_labels)

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    hub_model_id=model_name,
    output_dir="./output",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

In [ ]:
### Fine tune model
trainer.train()

In [ ]:
## Save pytorch 
trainer.save_model(model_name)

In [2]:
!pip install optimum[openvino,nncf] -q


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!optimum-cli export openvino --model intent --task text-classification intent_openvino/

In [3]:
!pip install essentia librosa


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [25]:
from transformers import AutoTokenizer
from optimum.intel import OVModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("intent_openvino")
model = OVModelForSequenceClassification.from_pretrained("intent_openvino")
inputs = tokenizer("I wanna finish my order", return_tensors="pt")
print("Inputs tensor",inputs)
with torch.no_grad():
    outputs = model(**inputs)
logits= outputs.logits
# Apply the softmax to get probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1)
# Get the predicted class label
predicted_classes = probabilities.argmax(dim=1).tolist()

predicted_classes

Compiling the model to CPU ...
Setting OpenVINO CACHE_DIR to intent_openvino/model_cache


Inputs tensor {'input_ids': tensor([[  101,  1045, 10587,  3926,  2026,  2344,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


[1]

In [ ]:
## Load the bucket and file path
bucket_name =  os.environ['bucket_name']
model_name = 'intent'

In [ ]:
## AWS_ACCESS_KEY_ID & AWS_SECRET_ACCESS_KEY should be set as Env variables
key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [ ]:
import boto3
from datetime import datetime

now_date = datetime.now()
s3_client = boto3.client('s3', aws_access_key_id=key_id, aws_secret_access_key=secret_key)

In [ ]:
#Upload file to S3
#s3_client.upload_file(optimized_onnx_filename, bucket_name, f"{model_path}/bins/{onnx_filename}")
#s3_client.upload_file(labeled_dataset, bucket_name,  f"{model_path}/datasets/dataset-{now_date.isoformat()}.csv")

In [ ]:
## Delete directory in Jupyter Notebook
import shutil

# Remove the local model directory
shutil.rmtree(model_name)
os.remove(optimized_onnx_filename)
os.remove(onnx_filename)
os.remove(labeled_dataset)
shutil.rmtree("output")